In [2]:
!pip install numpy PyPDF2 sentence-transformers chromadb colorama pandas tabulate

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached chromadb-1.1.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.2 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached pybase64-1.4.2-cp312-cp312-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (8.7 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.23.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.9 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.37.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 76.6 MB/s eta 0:00:00
   ━━━━

In [3]:
import os
import re
import warnings
from typing import List, Dict, Tuple
import pandas as pd
from tabulate import tabulate

import PyPDF2
import chromadb
from sentence_transformers import SentenceTransformer
from google.colab import files
warnings.filterwarnings('ignore')

In [4]:
# Try to import colorama for colored output
try:
    from colorama import init, Fore, Style
    init(autoreset=True)
    COLORS_AVAILABLE = True
except ImportError:
    COLORS_AVAILABLE = False
    class Fore:
        GREEN = CYAN = YELLOW = RED = MAGENTA = BLUE = WHITE = ""
    class Style:
        BRIGHT = RESET_ALL = ""

class ProductComparator:
    """
    A RAG-based chatbot to compare product features from uploaded brochures.
    Accepts queries like: "Compare battery and camera of iPhone vs Samsung"
    Outputs a clean comparison table.
    """

    def __init__(self, pdf_folder: str):
        self.pdf_folder = pdf_folder
        self.products = []  # List of {name, chunks}
        self.embedding_model = None
        self.collection = None
        self.client = None

        # Customize these based on your domain (phones, laptops, cars, etc.)
        self.feature_keywords = [
            'display', 'screen', 'battery', 'camera', 'processor', 'cpu', 'chip',
            'ram', 'memory', 'storage', 'price', 'cost', 'weight', 'dimensions',
            'os', 'operating system', 'connectivity', 'wifi', 'bluetooth', '5g',
            'warranty', 'color', 'material', 'water resistance', 'audio', 'speakers',
            'sensors', 'security', 'face id', 'fingerprint', 'charging', 'fast charge'
        ]

    def _extract_text_from_pdf(self, path: str) -> str:
        """Extract raw text from a PDF file."""
        try:
            with open(path, 'rb') as f:
                reader = PyPDF2.PdfReader(f)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() or ""
                return text
        except Exception as e:
            print(f"{Fore.RED}❌ Failed to read {path}: {e}{Style.RESET_ALL}")
            return ""

    def _chunk_by_features(self, text: str, product_name: str) -> List[Dict]:
        """Split brochure text into feature-based chunks."""
        chunks = []
        lines = text.split('\n')
        current_feature = "General"
        current_content = ""

        for line in lines:
            line_clean = line.strip()
            if not line_clean:
                continue

            # Detect feature headers (case-insensitive, short lines)
            matched_feature = None
            for feat in self.feature_keywords:
                if feat in line_clean.lower() and len(line_clean) < 80:
                    matched_feature = feat.title()
                    break

            if matched_feature:
                if current_content.strip():
                    chunks.append({
                        'product': product_name,
                        'feature': current_feature,
                        'content': current_content.strip()
                    })
                current_feature = matched_feature
                current_content = line_clean + "\n"
            else:
                current_content += line_clean + "\n"

        if current_content.strip():
            chunks.append({
                'product': product_name,
                'feature': current_feature,
                'content': current_content.strip()
            })
        return chunks

    def load_all_pdfs(self):
        """Load all PDFs from folder and chunk by features."""
        print(f"{Fore.CYAN}📂 Loading product brochures from: {self.pdf_folder}{Style.RESET_ALL}")
        if not os.path.exists(self.pdf_folder):
            raise FileNotFoundError(f"Folder not found: {self.pdf_folder}")

        pdf_files = [f for f in os.listdir(self.pdf_folder) if f.lower().endswith('.pdf')]
        if not pdf_files:
            raise FileNotFoundError("No PDF files found in the folder!")

        for pdf_file in pdf_files:
            product_name = os.path.splitext(pdf_file)[0].replace('_', ' ').replace('-', ' ').title()
            full_path = os.path.join(self.pdf_folder, pdf_file)
            text = self._extract_text_from_pdf(full_path)
            if not text.strip():
                print(f"{Fore.YELLOW}⚠️  Skipping empty PDF: {pdf_file}{Style.RESET_ALL}")
                continue
            chunks = self._chunk_by_features(text, product_name)
            self.products.append({'name': product_name, 'chunks': chunks})
            print(f"  ✅ Loaded: {product_name} ({len(chunks)} chunks)")

    def build_vector_store(self):
        """Build ChromaDB vector store with metadata."""
        print(f"{Fore.CYAN}🧠 Loading embedding model...{Style.RESET_ALL}")
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

        print(f"{Fore.CYAN}💾 Building vector database...{Style.RESET_ALL}")
        self.client = chromadb.Client()
        self.collection = self.client.create_collection("product_specs")

        all_docs, all_metadatas, all_ids = [], [], []
        chunk_id = 0

        for product in self.products:
            for chunk in product['chunks']:
                all_docs.append(chunk['content'])
                all_metadatas.append({
                    'product': chunk['product'],
                    'feature': chunk['feature']
                })
                all_ids.append(f"chunk_{chunk_id}")
                chunk_id += 1

        embeddings = self.embedding_model.encode(all_docs, show_progress_bar=False)
        self.collection.add(
            embeddings=embeddings.tolist(),
            documents=all_docs,
            metadatas=all_metadatas,
            ids=all_ids
        )
        print(f"{Fore.GREEN}✅ Database ready with {len(all_docs)} chunks{Style.RESET_ALL}")

    def is_comparison_query(self, query: str) -> bool:
        return any(word in query.lower() for word in ['compare', 'vs', 'versus', 'difference'])

    def extract_features_from_query(self, query: str) -> List[str]:
        """Extract relevant features mentioned in the query."""
        found = []
        query_lower = query.lower()
        for feat in self.feature_keywords:
            if feat in query_lower:
                # Use canonical name (first word)
                found.append(feat.split()[0].title())
        return list(dict.fromkeys(found))  # dedupe

    def retrieve_feature_value(self, product: str, feature: str) -> str:
        """Retrieve and clean the best value for a (product, feature) pair."""
        results = self.collection.query(
            query_texts=[f"{product} {feature} specifications"],
            n_results=1,
            where={"product": product, "feature": feature}
        )
        if results['documents'][0]:
            text = results['documents'][0][0]
            # Try to extract concise value: look for numbers + units
            match = re.search(r'(\d+\.?\d*\s*[a-zA-Z%]+(?:\s*/\s*\w+)?)', text)
            if match:
                return match.group(1)
            # Fallback: first 60 chars
            return (text[:60] + "...") if len(text) > 60 else text
        return "Not found"

    def generate_comparison_table(self, query: str) -> str:
        """Generate a markdown-style comparison table."""
        product_names = [p['name'] for p in self.products]
        features = self.extract_features_from_query(query)

        if not features:
            # Default to top common features if none specified
            features = ['Battery', 'Display', 'Camera', 'Processor', 'Price']

        data = {"Feature": features}
        for product in product_names:
            values = []
            for feat in features:
                val = self.retrieve_feature_value(product, feat)
                values.append(val)
            data[product] = values

        df = pd.DataFrame(data)
        return tabulate(df, headers='keys', tablefmt='grid', showindex=False)

    def setup(self) -> bool:
        """Full setup: load PDFs + build DB."""
        print(f"\n{Fore.CYAN}{Style.BRIGHT}╔{'═'*78}╗{Style.RESET_ALL}")
        print(f"{Fore.CYAN}{Style.BRIGHT}║{' '*18}🔍 COMPETITOR PRODUCT COMPARATOR{' '*22}║{Style.RESET_ALL}")
        print(f"{Fore.CYAN}{Style.BRIGHT}╚{'═'*78}╝{Style.RESET_ALL}\n")
        try:
            self.load_all_pdfs()
            self.build_vector_store()
            print(f"\n{Fore.GREEN}{Style.BRIGHT}┌{'─'*78}┐{Style.RESET_ALL}")
            print(f"{Fore.GREEN}│  ✅ Ready! Ask: 'Compare battery and camera of ProductA vs ProductB'{' '*5}│{Style.RESET_ALL}")
            print(f"{Fore.MAGENTA}│  🚪 Type 'quit' or 'exit' to stop{' '*45}│{Style.RESET_ALL}")
            print(f"{Fore.GREEN}{Style.BRIGHT}└{'─'*78}┘{Style.RESET_ALL}\n")
            return True
        except Exception as e:
            print(f"{Fore.RED}❌ Setup failed: {e}{Style.RESET_ALL}")
            return False

    def chat(self):
        """Interactive chat loop."""
        while True:
            try:
                query = input(f"{Fore.CYAN}{Style.BRIGHT}💬 You: {Style.RESET_ALL}").strip()
                if query.lower() in ['quit', 'exit', 'bye']:
                    print(f"\n{Fore.MAGENTA}👋 Goodbye! Thanks for comparing!{Style.RESET_ALL}\n")
                    break
                if not query:
                    continue

                if self.is_comparison_query(query):
                    table = self.generate_comparison_table(query)
                    print(f"\n{Fore.YELLOW}📊 Comparison Result:\n{Style.RESET_ALL}")
                    print(table)
                    print()
                else:
                    print(f"{Fore.RED}⚠️ Please ask a comparison question (e.g., 'Compare battery of X vs Y'){Style.RESET_ALL}\n")
            except KeyboardInterrupt:
                print(f"\n{Fore.MAGENTA}👋 Session ended.{Style.RESET_ALL}\n")
                break
            except Exception as e:
                print(f"{Fore.RED}❌ Error: {e}{Style.RESET_ALL}\n")

In [5]:
if __name__ == "__main__":
    # 👇 This section now handles file uploads

    print(f"{Fore.CYAN}Please upload your product brochures (PDFs):{Style.RESET_ALL}")
    uploaded = files.upload()

    temp_pdf_folder = "/content/uploaded_brochures"
    os.makedirs(temp_pdf_folder, exist_ok=True)

    # Save uploaded files to the temporary folder
    for fn, content in uploaded.items():
      print(f'Saving "{fn}"...')
      with open(os.path.join(temp_pdf_folder, fn), 'wb') as f:
        f.write(content)

    # Update the PDF_FOLDER variable to the temporary folder
    PDF_FOLDER = temp_pdf_folder

    print(f"\nUploaded files saved to: {PDF_FOLDER}")

    # Now proceed with the rest of the setup using the uploaded files
    comparator = ProductComparator(PDF_FOLDER)
    if comparator.setup():
        comparator.chat()
    else:
        print(f"\n{Fore.RED}⚠️  Initialization failed. Check uploaded files and PDFs.{Style.RESET_ALL}")

Please upload your product brochures (PDFs):


Saving BMW-Serie-3-2008-CN.pdf to BMW-Serie-3-2008-CN.pdf
Saving GMC-MotorHome-1973-USA.pdf to GMC-MotorHome-1973-USA.pdf
Saving "BMW-Serie-3-2008-CN.pdf"...
Saving "GMC-MotorHome-1973-USA.pdf"...

Uploaded files saved to: /content/uploaded_brochures

╔══════════════════════════════════════════════════════════════════════════════╗
║                  🔍 COMPETITOR PRODUCT COMPARATOR                      ║
╚══════════════════════════════════════════════════════════════════════════════╝

📂 Loading product brochures from: /content/uploaded_brochures
  ✅ Loaded: Bmw Serie 3 2008 Cn (1 chunks)
⚠️  Skipping empty PDF: GMC-MotorHome-1973-USA.pdf
🧠 Loading embedding model...
💾 Building vector database...
✅ Database ready with 1 chunks

┌──────────────────────────────────────────────────────────────────────────────┐
│  ✅ Ready! Ask: 'Compare battery and camera of ProductA vs ProductB'     │
│  🚪 Type 'quit' or 'exit' to stop                                             │
└─────────────────────────